In [23]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from keras.preprocessing.image import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [24]:
train_gen = image_dataset_from_directory(directory="../input/tomatoleaf/tomato/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="../input/tomatoleaf/tomato/val",
                                        image_size=(256, 256))

rescale = Rescaling(scale=1.0/255)

train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 10000 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers
import tensorflow as tf
#from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K

In [26]:
Model = Sequential()

Model.add(SeparableConv2D(32,3,
                          activation="relu",
                 input_shape=(220,220,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2)))

#
Model.add(SeparableConv2D(64,3,
                 activation="relu"))
Model.add(SeparableConv2D(128,(3,3),
                 activation="relu"))
Model.add(Dropout(0.25))
Model.add(MaxPooling2D((2)))

#
Model.add(SeparableConv2D(128,3,
                 activation="relu"))
Model.add(SeparableConv2D(256,3,
                 activation="relu"))
Model.add(Dropout(0.25))
Model.add(MaxPooling2D((2)))
#
Model.add(SeparableConv2D(256,3,
                 activation="relu"))
Model.add(SeparableConv2D(512,3,
                 activation="relu"))
Model.add(Dropout(0.25))
Model.add(GlobalAveragePooling2D())

#
Model.add(Flatten())
Model.add(Dense(1024,
                activation="relu"))
Model.add(Dropout(0.25))
Model.add(Dense(10,
                activation="softmax"))

Model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_15 (Separab (None, 218, 218, 32)      155       
_________________________________________________________________
batch_normalization_3 (Batch (None, 218, 218, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 109, 109, 32)      0         
_________________________________________________________________
separable_conv2d_16 (Separab (None, 107, 107, 64)      2400      
_________________________________________________________________
separable_conv2d_17 (Separab (None, 105, 105, 128)     8896      
_________________________________________________________________
dropout_9 (Dropout)          (None, 105, 105, 128)     0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 52, 52, 128)      

In [ ]:
Model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=["accuracy"])
#aug = ImageDataGenerator(fill_mode="nearest")
history = Model.fit_generator(train_gen,
        validation_data=test_gen,
        epochs=100, verbose=1
)  


Epoch 1/100
313/313 [==============================] - 101s 321ms/step - loss: 1.8187 - accuracy: 0.2917 - val_loss: 2.1367 - val_accuracy: 0.1960
Epoch 2/100
313/313 [==============================] - 101s 321ms/step - loss: 1.2062 - accuracy: 0.5591 - val_loss: 1.5338 - val_accuracy: 0.4350
Epoch 3/100
313/313 [==============================] - 101s 322ms/step - loss: 0.7854 - accuracy: 0.7206 - val_loss: 1.0649 - val_accuracy: 0.5860
Epoch 4/100
313/313 [==============================] - 101s 321ms/step - loss: 0.5888 - accuracy: 0.7970 - val_loss: 0.6410 - val_accuracy: 0.8000
Epoch 5/100
313/313 [==============================] - 101s 322ms/step - loss: 0.4629 - accuracy: 0.8396 - val_loss: 0.5896 - val_accuracy: 0.8230
Epoch 6/100
313/313 [==============================] - 101s 323ms/step - loss: 0.3951 - accuracy: 0.8601 - val_loss: 0.3824 - val_accuracy: 0.8850
Epoch 7/100
313/313 [==============================] - 101s 322ms/step - loss: 0.3670 - accuracy: 0.8683 - val_loss: 0

In [ ]:
EPOCHS=100
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.title("Train and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(range(EPOCHS),history.history['loss'],label="Train Loss")
plt.plot(range(EPOCHS),history.history['val_loss'], label="Validation Loss")
plt.legend()

plt.subplot(1,2,2)
plt.title("Train and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.plot(range(EPOCHS),history.history['accuracy'], label="Train Accuracy")
plt.plot(range(EPOCHS),history.history['val_accuracy'], label="Validation Accuracy")
plt.legend()
plt.tight_layout()


In [ ]:
labels = []
predictions = []
for x,y in test_gen:
    labels.append(list(y.numpy()))
    predictions.append(tf.argmax(Model.predict(x),1).numpy())

In [ ]:
predictions = list(itertools.chain.from_iterable(predictions))
labels = list(itertools.chain.from_iterable(labels))

In [ ]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(labels, predictions) * 100))
print("Precision Score : {:.2f} %".format(precision_score(labels, predictions, average='micro') * 100))
print("Recall Score    : {:.2f} %".format(recall_score(labels, predictions, average='micro') * 100))

In [ ]:
cm = confusion_matrix(labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=list(range(1,11)))
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax)
plt.title("Confusion Matrix")
plt.show()